<a href="https://colab.research.google.com/github/Rupak-Mallick/-_-basic-calculator/blob/main/Writing_Assistant_with_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install required packages
!pip install -q -U langchain langchain-groq langchain-community python-dotenv gradio

# ✅ Imports
import os
from google.colab import userdata
import gradio as gr
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.tools import tool

# ✅ Load API key from Colab secrets
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
except Exception as e:
    raise ValueError("⚠️ Failed to load Groq API Key from Colab secrets")

# ✅ Writing Knowledge Base
WRITING_KNOWLEDGE = {
    "hook": "A hook is a compelling first sentence designed to grab the reader's attention.",
    "thesis": "A thesis statement clearly expresses the main idea of your paper.",
    "conclusion": "A strong conclusion summarizes your argument and reinforces your thesis."
}

# ✅ Tool 1: Writing Knowledge Base
@tool
def writing_knowledge_base(topic: str) -> str:
    """Retrieve writing-related tips or definitions from knowledge base"""
    topic = topic.lower().strip()
    if topic in WRITING_KNOWLEDGE:
        return WRITING_KNOWLEDGE[topic]
    for key in WRITING_KNOWLEDGE:
        if key in topic:
            return WRITING_KNOWLEDGE[key]
    return ""

# ✅ Tool 2: Reading Time Calculator
@tool
def estimate_reading_time(words: str) -> str:
    """Calculate exact reading time from word count"""
    try:
        word_count = int(''.join(filter(str.isdigit, words)))
        if word_count <= 0:
            return "Word count must be positive"
        minutes = word_count / 200
        if minutes < 1:
            return f"Exact reading time: {int(word_count / 200 * 60)} seconds ({word_count} words)"
        elif minutes > 60:
            hours = int(minutes // 60)
            remaining_mins = int(minutes % 60)
            return f"Exact reading time: {hours}h {remaining_mins}m ({word_count} words)"
        else:
            return f"Exact reading time: {minutes:.1f} minutes ({word_count} words)"
    except:
        return "Please enter like: 1200 or '1200 words'"

# ✅ Tools initialization
tools = [
    Tool(name="WritingGuide", func=writing_knowledge_base, description="For writing terms like hook, thesis, conclusion"),
    Tool(name="ReadingTime", func=estimate_reading_time, description="For calculating reading time from word count")
]

# ✅ Prompt Template
writing_prompt = PromptTemplate(
    input_variables=["task"],
    template="""
You are a professional writing assistant. Provide short, clear, and helpful responses in this format:

Definition (1 sentence)
- Bullet point 1
- Bullet point 2

Only include an example if it's very helpful.

Examples:

Q: What is a thesis statement?
A: A thesis statement summarizes the main argument of your essay in one sentence.
- Guides the essay's structure and focus
- Should be clear, specific, and arguable

Q: What is a hook in writing?
A: A hook is the opening sentence that grabs the reader’s attention.
- Can be a question, quote, or bold statement
- Makes the reader want to keep reading

Q: {task}
A:"""
)

# ✅ Initialize LLM
llm = ChatGroq(
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7,
    groq_api_key=groq_api_key
)

# ✅ Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# ✅ Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=False,
    max_iterations=2,
    early_stopping_method="generate",
    handle_parsing_errors=True
)

# ✅ Response function for Gradio
def writing_assistant(user_input):
    if not user_input.strip():
        return "❌ Please enter a question."

    # Tool 1: Knowledge Base
    kb_answer = writing_knowledge_base.invoke(user_input)
    if kb_answer and kb_answer.strip() != "":
        return f"🛠️ Used Knowledge Base:\n\n{kb_answer}"

    # Tool 2: Reading Time
    if any(word in user_input.lower() for word in ["word", "read", "minute", "hour", "'", "\""]):
        nums = []
        for word in user_input.split():
            clean_word = word.strip('"\'').replace(',', '')
            if clean_word.isdigit():
                nums.append(int(clean_word))
        if nums:
            return f"🛠️ Used Reading Calculator:\n\n{estimate_reading_time.invoke(str(nums[0]))}"

    # Else use LLM
    try:
        response = agent.invoke({
            "input": writing_prompt.format(task=user_input)
        })
        return f"🤖 Used LLM:\n\n{response['output']}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# ✅ Gradio UI
gr.Interface(
    fn=writing_assistant,
    inputs=gr.Textbox(label="✍️ Enter your question", placeholder="e.g., What is a thesis statement?"),
    outputs=gr.Textbox(label="🧠 Assistant Response"),
    title="📝 AI Writing Assistant",
    description="Ask about writing techniques, terms, or reading time. Powered by LangChain, Groq & LLaMA 4."
).launch()


<ipython-input-1-20d19b5a6ef3>:101: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-1-20d19b5a6ef3>:104: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://197cf43112d9a17f8b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
